# training t5 for chinese news summery


In [10]:
# data load
import pandas as pd    
data_sample_path = "/data/agent_h/news2016zh/news2016zh_valid.json"
data = pd.read_json(path_or_buf=data_sample_path, lines=True)
data = data.to_dict('records')

76797


In [29]:
# data explore
from pprint import pprint
import random
print(len(data))
idx = int(random.random()*len(data))
print("id: ", idx)
pprint(data[idx])
print("title: ", data[idx]['title'])
print("content: ", data[idx]['content'][:512])

76797
id:  14784
{'content': '近日，世界贸易组织(WTO)对中国第六次贸易政策审议在日内瓦顺利结束。此次审议过程中，中国经贸体制、贸易投资领域取得的新进展等多方面得到积极评价，各成员对中国成为其重要经贸合作伙伴十分重视。对此，专家指出，虽然上半年中国进出口双下降，但中国在全球贸易经济中的地位仍不断上升，尤其是中国外贸新旧动能转换释放出的强劲动力，将推动对外贸易继续回稳、向好，也将为全球贸易增长作出重要贡献。新动能持续积累优势上半年，我国进出口同比下降3.3%，进口、出口分别下降4.7%和2.1%。虽然进出口双下降，但我国外贸新旧动能转换正加快进程，贸易结构不断优化。海关总署数据显示，1-6月，我国一般贸易进出口占进出口总值的56.4%，比去年同期提升1.2个百分点;民企出口增长3.6%，占出口总值的46.6%，占比继续保持首位。“一般贸易占比的持续上升体现出中国自主产品的比重在上升、自主创新能力在增强，我国对外贸易正向高附加值端发展，如高新技术产业等新业态在我国外贸发展中的势头已越来越强劲。而民企出口的快速发展带来了更多活力，外贸中的国内资本和投入品的增加，为我国外贸健康发展及结构优化提供了新动能。”国家发改委对外经济研究所国际合作室主任张建平在接受本报记者采访时说。多边、双边经贸合作不断拓展则为我国外贸提供了更大发展空间。海关总署新闻发言人黄颂平指出，上半年，我国对部分“一带一路”沿线国家出口增长。另外，已有22个国家或地区与我国签署并实施自贸协定，上半年，与上述国家或地区的进出口表现好于同期我国进出口总体降幅。商务部研究院国际市场研究部副主任白明表示，上半年，大型成套产品出口保持正增长，这个领域的商品技术含量高，附加值也比较高，跟一般的传统商品相比，它更是我们发展的一个方向。跨境电商贸易、市场采购贸易等新型外贸商业模式正成为新的外贸增长点。中国贸促会副会长尹宗华指出，去年我国的跨境电子商务规模为5.4万亿元人民币，预计今年可能会达到6.5万亿元人民币，对于促进外贸稳增长、调结构发挥了重要作用。“机器换人”降低成本随着新旧动能转换的持续推进，传统动能这一曾经的外贸主要贡献者正面临困境。海关总署数据显示，截至今年6月份，我国的加工贸易进口、出口已经分别连续18个月和16个月下降。今年上半年，加工贸易进出口下降9.8%，拖累我国外贸进出口

In [3]:
# end exploring, from now on this is in acutal training script
# create dataset
# TODO test sharded
import pandas as pd    
import transformers
from datasets import load_dataset, load_metric, Dataset
raw_data_train_path = "/data/agent_h/news2016zh/news2016zh_train.json"
raw_data_valid_path = "/data/agent_h/news2016zh/news2016zh_valid.json"

USE_SAMPLE_DATASET = False # use small set 
if USE_SAMPLE_DATASET:
    train_data = pd.read_json(path_or_buf=raw_data_valid_path, lines=True)
    train_data = train_data.to_dict('records')
    train_data = train_data[:]
else:
    train_data = pd.read_json(path_or_buf=raw_data_train_path, lines=True)
    train_data = train_data.to_dict('records')

print("train size: ", len(train_data))

train_data = Dataset.from_list(train_data)
train_data = train_data.train_test_split(test_size=min(5000,int(len(train_data)*0.1)))


train size:  2430752


In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def preprocess_cn_news_text_data(dataset,
                                 tokenizer_base = "/data/agent_h/llms/umt5-small",
                                 data_key = 'content',
                                 label_key = 'title',
                                 prefix="生成标题：",
                                 max_input_len=512,
                                 max_target_len=64,
                                ):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_base)
    inputs = [prefix + text for text in dataset[data_key]]
    inputs = tokenizer(inputs,
                       max_length=max_input_len,
                       truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(dataset[label_key],
                           max_length=max_input_len, 
                           truncation=True)
    
    inputs["labels"] = labels["input_ids"]
    return inputs

# TODO how to save this thing

model_base = "/data/agent_h/llms/umt5-small"
dataset_path = "/data/agent_h/umt5-small-news2016zh-tokens-full"
# tokenize
train_data = train_data.map(preprocess_cn_news_text_data,batched=True,batch_size=100000)
train_data.save_to_disk(dataset_path)


Map:   0%|                                                                     | 0/2425752 [00:00<?, ? examples/s]/home/agent_h/miniconda3/envs/vsr3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|█████████████████████████████| 5000/5000 [00:00<00:00, 246078.17 examples/s]


In [33]:
# training in 11_t5_exp2.py